In [2]:
# ==========================================
# Setup and Imports
# ==========================================

import sys
sys.path.append('..')

import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter

# Import data utilities
from src.data_utils import load_dataset

print("Imports complete!")

Imports complete!


In [3]:
# ==========================================
# Load Dataset
# ==========================================

dataset = load_dataset('../data/processed/names_group3.txt')

print(f"Dataset loaded: {len(dataset.train_names):,} training names")
print(f"Vocabulary size: {dataset.vocab_size}")

Loaded 102,450 names from names_group3.txt
Vocabulary size: 27 characters
   Characters: abcdefghijklmnopqrstuvwxyz
Dataset splits:
   Train: 81,960 names (80%)
   Val:   10,245 names (10%)
   Test:  10,245 names (10%)

DATASET STATISTICS
Total names:    102,450
Train names:    81,960
Val names:      10,245
Test names:     10,245
Vocabulary:     27 characters
Name length:    min=2, max=15, avg=6.5

Dataset loaded: 81,960 training names
Vocabulary size: 27


In [4]:
# ==========================================
# Name Generation Function
# ==========================================

@torch.no_grad()
def generate_name(max_length=20):
    """Generate a name using the trained WaveNet model."""
    name = []
    context = [dataset.stoi['.']] * block_size
    
    while True:
        # Prepare input
        x = torch.tensor([context])
        
        # Forward pass
        emb = C[x]
        embcat = emb.view(1, -1)
        
        # Layer 1
        hpreact1 = embcat @ W1 + b1
        hpreact1 = bngain1 * (hpreact1 - bnmean_running1) / bnstd_running1 + bnbias1
        h1 = torch.tanh(hpreact1)
        
        # Layer 2
        hpreact2 = h1 @ W2 + b2
        hpreact2 = bngain2 * (hpreact2 - bnmean_running2) / bnstd_running2 + bnbias2
        h2 = torch.tanh(hpreact2)
        
        # Layer 3
        hpreact3 = h2 @ W3 + b3
        hpreact3 = bngain3 * (hpreact3 - bnmean_running3) / bnstd_running3 + bnbias3
        h3 = torch.tanh(hpreact3)
        
        # Output
        logits = h3 @ Wout + bout
        probs = F.softmax(logits, dim=1)
        
        # Sample next character
        ix = torch.multinomial(probs, num_samples=1).item()
        
        # Stop at period
        if ix == dataset.stoi['.']:
            break
        
        # Add to name
        name.append(dataset.itos[ix])
        
        # Update context
        context = context[1:] + [ix]
        
        if len(name) >= max_length:
            break
    
    return ''.join(name)

print("Generation function ready!")

Generation function ready!


In [ ]:
# ==========================================
# Load Best Model (WaveNet block_size=10)
# ==========================================

# Hyperparameters (must match training)
block_size = 10
n_embd = 10
n_hidden = 200

print("Loading WaveNet model (block_size=10)...")

# Load model checkpoint
checkpoint = torch.load('../outputs/wavenet_block10_model.pth')

# Extract parameters
C = checkpoint['C']
W1 = checkpoint['W1']
b1 = checkpoint['b1']
bngain1 = checkpoint['bngain1']
bnbias1 = checkpoint['bnbias1']
bnmean_running1 = checkpoint['bnmean_running1']
bnstd_running1 = checkpoint['bnstd_running1']

W2 = checkpoint['W2']
b2 = checkpoint['b2']
bngain2 = checkpoint['bngain2']
bnbias2 = checkpoint['bnbias2']
bnmean_running2 = checkpoint['bnmean_running2']
bnstd_running2 = checkpoint['bnstd_running2']

W3 = checkpoint['W3']
b3 = checkpoint['b3']
bngain3 = checkpoint['bngain3']
bnbias3 = checkpoint['bnbias3']
bnmean_running3 = checkpoint['bnmean_running3']
bnstd_running3 = checkpoint['bnstd_running3']

Wout = checkpoint['Wout']
bout = checkpoint['bout']

print("Model loaded successfully!")
print(f"   Val Loss: {checkpoint.get('val_loss', 'N/A')}")
print(f"   Parameters: {checkpoint.get('total_params', 'N/A'):,}")

In [ ]:
# ==========================================
# REAL-TIME NAME GENERATION
# ==========================================

import time

print("\n" + "=" * 80)
print("REAL-TIME NAME GENERATION DEMO")
print("=" * 80)
print("\nWatch the model generate names character-by-character!\n")

def generate_name_with_display(max_length=20):
    """Generate name and display character-by-character."""
    name = []
    context = [dataset.stoi['.']] * block_size
    
    print("   Generating: ", end="", flush=True)
    
    while True:
        x = torch.tensor([context])
        
        # Forward pass
        emb = C[x]
        embcat = emb.view(1, -1)
        
        hpreact1 = embcat @ W1 + b1
        hpreact1 = bngain1 * (hpreact1 - bnmean_running1) / bnstd_running1 + bnbias1
        h1 = torch.tanh(hpreact1)
        
        hpreact2 = h1 @ W2 + b2
        hpreact2 = bngain2 * (hpreact2 - bnmean_running2) / bnstd_running2 + bnbias2
        h2 = torch.tanh(hpreact2)
        
        hpreact3 = h2 @ W3 + b3
        hpreact3 = bngain3 * (hpreact3 - bnmean_running3) / bnstd_running3 + bnbias3
        h3 = torch.tanh(hpreact3)
        
        logits = h3 @ Wout + bout
        probs = F.softmax(logits, dim=1)
        
        ix = torch.multinomial(probs, num_samples=1).item()
        
        if ix == dataset.stoi['.']:
            break
        
        char = dataset.itos[ix]
        name.append(char)
        print(char, end="", flush=True)
        time.sleep(0.1)  # Dramatic pause
        
        context = context[1:] + [ix]
        
        if len(name) >= max_length:
            break
    
    print(" ")
    return ''.join(name)

# Generate 10 names with animation
print("Generating 10 names in real-time...\n")
demo_names = []
for i in range(10):
    print(f"{i+1:2d}.", end=" ")
    name = generate_name_with_display()
    demo_names.append(name)
    time.sleep(0.3)

print("\n" + "=" * 80)
print("Real-time generation complete!")
print("=" * 80)


REAL-TIME NAME GENERATION DEMO

Watch the model generate names character-by-character!

🎲 Generating 10 names in real-time...

 1.    Generating: finnadeyas 
 2.    Generating: teiden 
 3.    Generating: desie 
 4.    Generating: youn 
 5.    Generating: ramon 
 6.    Generating: gurd 
 7.    Generating: guci 
 8.    Generating: lexe 
 9.    Generating: melia 
10.    Generating: aylen 

Real-time generation complete!


In [ ]:
# ==========================================
# BATCH GENERATION (30 Names)
# ==========================================

print("\nGenerating 30 additional names quickly...\n")

more_names = []
for i in range(30):
    name = generate_name()
    more_names.append(name)
    print(f"{i+1:2d}. {name:<20}", end="")
    if (i + 1) % 3 == 0:
        print()

print("\n" + "=" * 80)
print(f"Generated {len(more_names)} more names!")
print("=" * 80)


🎲 Generating 30 additional names quickly...

 1. lataria              2. kenisa               3. nessicit            
 4. cathcia              5. coorin               6. brishaet            
 7. risha                8. caveettt             9. sevir               
10. brit                11. izmon               12. viombel             
13. jaissa              14. miann               15. naraabeea           
16. tsofana             17. berneven            18. amyaen              
19. abjan               20. couren              21. bashe               
22. indye               23. gurika              24. clemanis            
25. bawleng             26. devei               27. brittaymi           
28. arlangede           29. arighett            30. johneve             

✅ Generated 30 more names!


In [11]:
# ==========================================
# GENERATION STATISTICS
# ==========================================

import numpy as np
from collections import Counter

# Combine all generated names

print("\nGENERATION STATISTICS")
print("=" * 80)

name_lengths = [len(name) for name in demo_names]

print(f"\nTotal names generated: {len(demo_names)}")
print(f"Average length: {np.mean(name_lengths):.1f} characters")
print(f"Shortest: {min(name_lengths)} chars ('{min(demo_names, key=len)}')")
print(f"Longest: {max(name_lengths)} chars ('{max(demo_names, key=len)}')")
print(f"Unique names: {len(set(demo_names))}/{len(demo_names)}")

# Length distribution
print(f"\nLength Distribution:")
length_counts = Counter(name_lengths)
for length in sorted(length_counts.keys()):
    bar = "█" * length_counts[length]
    print(f"  {length:2d} chars: {bar} ({length_counts[length]})")

print("=" * 80)


GENERATION STATISTICS

Total names generated: 10
Average length: 5.2 characters
Shortest: 4 chars ('youn')
Longest: 10 chars ('finnadeyas')
Unique names: 10/10

Length Distribution:
   4 chars: ████ (4)
   5 chars: ████ (4)
   6 chars: █ (1)
  10 chars: █ (1)


In [13]:
# ==========================================
# SAVE DEMO NAMES
# ==========================================

# Get values from checkpoint (loaded earlier)
val_loss = checkpoint['val_loss']
total_params = checkpoint['total_params']

# Save to file
output_file = '../outputs/demo_generated_names.txt'
with open(output_file, 'w') as f:
    f.write("makemore - Generated Names Demo\n")
    f.write("=" * 80 + "\n\n")
    f.write(f"Model: WaveNet (block_size={block_size})\n")
    f.write(f"Val Loss: {val_loss:.4f}\n")
    f.write(f"Parameters: {total_params:,}\n\n")
    f.write("Generated Names:\n")
    f.write("-" * 80 + "\n\n")
    
    for i, name in enumerate(demo_names, 1):
        f.write(f"{i:3d}. {name}\n")

print(f"\nSaved {len(demo_names)} names to: {output_file}")
print("=" * 80)


Saved 10 names to: ../outputs/demo_generated_names.txt
